In [ ]:
using Pkg

#Pkg.add("DataFrames")
#Pkg.add("Plots")
#Pkg.add("CSV")

import DataFrames
import Plots
import CSV

In [ ]:

df = CSV.read("../dat/2024-01-23_14-15-08/output.csv", DataFrames.DataFrame)
#print(df)

Plots.scatter(
    df.T_total,
    df.V_Max,
    title = "Validation scores",
    ylabel = "V_Max",
    xlabel = "T_total",
)
#Plots.savefig("t3validation.pdf")

In [ ]:
pwd()
#cd("../logs/2024-02-05_15-37-53/")
pwd()
#ls()
stat("training_data.csv")

In [ ]:
cd("scripts")

In [ ]:

print(pwd())
df = CSV.read("../logs/2024-02-05_15-37-53/training_data.csv", DataFrames.DataFrame)
#print(df)

Plots.scatter(
    df.T_total,
    df.V_Max,
    title = "Validation scores",
    ylabel = "V_Max",
    xlabel = "T_total",
)

In [ ]:

print(pwd())
df = CSV.read("../logs/2024-02-05_15-37-53/training_data.csv", DataFrames.DataFrame)

Plots.scatter(
    df.T_total,
    df.T_Max,
    title = "Training scores",
    ylabel = "T_Max",
    xlabel = "T_total",
)

In [ ]:
function training_data_analysis(path_file)
    df = CSV.read("../logs/2024-02-05_15-37-53/training_data.csv", DataFrames.DataFrame)

    Plots.scatter(
        df.T_total,
        df.T_Max,
        title = "Training scores",
        ylabel = "T_Max",
        xlabel = "T_total",
    )

In [ ]:
training_data_file = "training_data.csv"

path_log_directories = readdir(abspath("../logs"), join=true, sort=false) 
for path_log_directory in path_log_directories
    #println(path_log_directory)
    content_log_directory = readdir(abspath(path_log_directory))
    #print(typeof(content_log_directory))
    if training_data_file in content_log_directory
        training_data_analysis(path_log_directory * "/" * training_data_file)
        validation_data_analysis(path_log_directory * "/" * training_data_file)
    else 
        println("File not found: training_data.csv in folder:" * path_log_directory)
    end
end

In [ ]:
df = CSV.read("test.csv", DataFrames.DataFrame)
for i, m in df.groupby("Measurement")
    ax.plot(m.Time, m.Mean)
    ax.fill_between(m.Time, m.Mean - m.Std, m.Mean + m.Std, alpha=0.35)

In [ ]:
using Plots

xs = range(1,100, step=1)
println(xs)
μs = log.(xs)
println(μs)

σs = rand(length(xs))
println(σs)

plot(xs,μs,grid=false,ribbon=σs,fillalpha=.5)

In [ ]:
print(rand(2))

In [ ]:
using Plots
using CSV
using DataFrames

# Read the data from the CSV file
df = CSV.read("../logs/2024-02-05_15-37-53/training_data.csv", DataFrame)

# Extract relevant columns
Gen = df.Gen
T_Avg = df.T_Avg
T_Max = df.T_Max
T_Min = df.T_Min

# Plotting the data
p = plot(Gen, T_Avg, xlabel="Generation", 
ylabel="Average Fitness", 
label="Average Fitness over Training", grid=false, 
legend=:bottomright, lw=3)
xlims!(0, maximum(Gen))
ylims!(minimum(T_Min), 0)

print(df)

In [ ]:
x = collect(range(0, 2, length= 100))
y1 = exp.(x)
y2 = exp.(1.3 .* x)

plot(x, y1, fillrange = y2, fillalpha = 0.35, c = 1, label = "Confidence band", legend = :topleft)

In [ ]:
plot!(x,y1, line = :scatter, c = 1, msw = 0, ms = 1.0, label = "Lower bound")
plot!(x,y2, line = :scatter, c = 1, msw = 0, ms = 1.0, label = "Upper bound")

In [ ]:
using Plots
using CSV
using DataFrames


# Create data frames array from CSV files of the different seeds 
dfs = [ 
    CSV.read("../logs/2024-02-05_15-37-53_copy/training_data.csv", DataFrame),
    CSV.read("../logs/2024-02-05_15-37-53_copy/fake_training_data_lower.csv", DataFrame),
    CSV.read("../logs/2024-02-05_15-37-53_copy/fake_training_data_higher.csv", DataFrame) 
    ]

# Check that the number of Gen is the same in each file 
Gen_ref = dfs[1].Gen
for e in dfs
    if e.Gen != Gen_ref
        throw(error("number of Generations is not the same in each data file"))
    end
end

# Extract the relevant columns, depends on the needs. e.g. Validation Max
relevant_data = []
for e in dfs
    push!(relevant_data, e[!, "V_Max"])
end

# for each seed, the last Gen is considered the reference score value
last_scores = [] 
for e in relevant_data
    push!(last_scores, e[Gen_ref[end]])
end

score_max, i_max = findmax(last_scores)
score_min, i_min = findmin(last_scores)

mean_scores = []
# for each generation
for i in Gen_ref[begin]:Gen_ref[end]
    # find the mean score of the current generation
    mean_i = 0
    for e in relevant_data
        mean_i += e[i+1] 
    end 
    mean_i /= length(relevant_data)
    push!(mean_scores, mean_i)
end


# Plotting the mean of the data
p = plot(Gen_ref, mean_scores, xlabel="Gen", ylabel="Score",  
grid=false, legend=:bottomright, lw=2)
xlims!(0, maximum(Gen_ref))
#ylims!(minimum(mean_scores), 0)

# Plotting lowest and highest TPG with shade btw them
plot!(Gen_ref, relevant_data[i_min], fillrange = relevant_data[i_max], fillalpha = 0.35, c = 1, 
lw = 0, label = "Confidence band", legend = :topleft)
plot!(Gen_ref, relevant_data[i_min], line = :scatter, c = 1, msw = 0, ms = 1.0, label = "Lower bound")
plot!(Gen_ref, relevant_data[i_max], line = :scatter, c = 1, msw = 0, ms = 1.0, label = "Upper bound")

### Compare multiple curves ###
compare time over nbGen to see which training is the longest